In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
39,application_1607211657348_0041,pyspark,idle,Link,Link


SparkSession available as 'spark'.

## Placeholder

In [2]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

ben_td = fs.get_training_dataset("gan_non_sar_training_df", 1)
eval_td = fs.get_training_dataset("gan_eval_df", 1)

Connected. Call `.close()` to terminate connection gracefully.

In [9]:
train_input = ben_td.tf_data(target_name='target', is_training=True)
train_input.tf_record_dataset(process=True, batch_size=16, num_epochs=1)

<PrefetchDataset shapes: ((16,), (16,)), types: (tf.string, tf.float32)>

## Placeholder

In [7]:
def experiment_wrapper():

    import tensorflow as tf
    from model.gan_enc_ano import GanAnomalyDetector,  GanAnomalyMonitor 
    from hops import tensorboard

    # Set the number of epochs for trainining.
    EPOCHS = 2

    train_input = ben_td.tf_data(target_name='target', is_training=True)
    train_input_processed = train_input.tf_record_dataset(process=True, batch_size=16, num_epochs=EPOCHS)
    eval_input = ben_td.tf_data(target_name='target', is_training=True)
    eval_input_processed = eval_input.tf_record_dataset(process=True, batch_size=1, num_epochs=EPOCHS)    

    # Instantiate the WGAN model.
    gan_anomaly_detector = GanAnomalyDetector(

                input_dim=16,
                latent_dim=4,

                discriminator_start_n_units=16,
                discriminator_n_layers=2,
                discriminator_activation_fn="sigmoid",
                discriminator_double_neurons=False,
                discriminator_bottleneck_neurons=True,
                discriminator_batch_norm=False,
                discriminator_batch_dropout=False,
                discriminator_dropout_rate=0.0,
                discriminator_learning_rate=0.0002,
                discriminator_extra_steps = 3,

                generator_start_n_units=8,
                generator_n_layers=2,
                generator_activation_fn="tanh",
                generator_double_neurons=True,
                generator_bottleneck_neurons=False,
                generator_batch_norm=False,
                generator_batch_dropout=False,
                generator_dropout_rate=0.0,
                generator_learning_rate=0.0002,

                encoder_start_n_units=4,
                encoder_n_layers=2,
                encoder_activation_fn="tanh",
                encoder_batch_norm=False,
                encoder_batch_dropout=False,
                encoder_dropout_rate=0.0,
                encoder_learning_rate=0.0002,

    )
    
    # Compile the WGAN model.
    gan_anomaly_detector.compile()

    callbacks = [
        GanAnomalyMonitor(batch_size=1, latent_dim=4, input_dim=16, alpha=0.7, real_data=eval_input_processed),
        tf.keras.callbacks.TensorBoard(log_dir=tensorboard.logdir()),
        tf.keras.callbacks.ModelCheckpoint(filepath=tensorboard.logdir()),
    ]
    # Start training the model.
    history = gan_anomaly_detector.fit(train_input_processed, callbacks=[callbacks])
    
    return history["g_loss"]


In [8]:
from hops import experiment
# experiment.collective_all_reduce(main)
experiment.launch(experiment_wrapper)

An error was encountered:
An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 9, hopsworks0.logicalclocks.com, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 2499, in pipeline_func
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 2499, in pipeline_func
  File "/srv/hops/spark/python/lib/pyspark.zip/pyspark/rdd.py", line 2499, in pipeline_func
  [Previous line repeated 1 more time]
  File "/srv/hops/spark/python/lib